In [1]:
from pyspark.sql.functions import col

StatementMeta(, 1d9abe8a-9a97-4ee6-aac5-88c9b0d9c653, 3, Finished, Available, Finished)

In [2]:
df = spark.sql("SELECT * FROM FabricDemoWorkspace.bing_lake_db.dbo.tbl_latest_news")
display(df)

StatementMeta(, 1d9abe8a-9a97-4ee6-aac5-88c9b0d9c653, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7860ebb3-1c19-4e63-9a97-bce761ba64ac)

In [3]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 1d9abe8a-9a97-4ee6-aac5-88c9b0d9c653, 5, Finished, Available, Finished)

In [7]:
model = (AnalyzeText()
        .setTextCol("title")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))


StatementMeta(, 1d9abe8a-9a97-4ee6-aac5-88c9b0d9c653, 9, Finished, Available, Finished)

In [8]:
result = model.transform(df)
display(result)

StatementMeta(, 1d9abe8a-9a97-4ee6-aac5-88c9b0d9c653, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0d95488f-046b-4951-a004-ddc745f329a4)

In [9]:
sentiment_df = result.withColumn("sentiment",col("response.documents.sentiment")).drop("response","error","position")
display(sentiment_df)

StatementMeta(, 1d9abe8a-9a97-4ee6-aac5-88c9b0d9c653, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 50d3867d-bd6b-43f6-a902-485b8ec2f876)

In [11]:
from pyspark.sql.utils import AnalysisException
try:
    table_name = "tbl_sentiment_analysis"
    sentiment_df.write.format("delta").saveAsTable(table_name)
    print("Table created")
except AnalysisException:
    print("Table already exists")
    sentiment_df.createOrReplaceTempView("vw_sentiment_analysis")
    spark.sql(f"""MERGE INTO {table_name} tgt USING vw_sentiment_analysis src
                ON src.link = tgt.link
                WHEN MATCHED AND
                src.title <> tgt.title OR
                src.date <> tgt.date OR
                src.source <> tgt.source OR
                src.icon <> tgt.icon
                THEN UPDATE SET *
                WHEN NOT MATCHED
                THEN INSERT *
            """)

StatementMeta(, 1d9abe8a-9a97-4ee6-aac5-88c9b0d9c653, 13, Finished, Available, Finished)

Table already exists


StatementMeta(, 1d9abe8a-9a97-4ee6-aac5-88c9b0d9c653, 15, Finished, Available, Finished)

NameError: name '__sqldf' is not defined